In [9]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging

import matplotlib.pyplot as plt

In [10]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

In [61]:
def dta_to_candlestick(data):
    l = len(data)
    # Make candlestick picture
    layout = go.Layout(xaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False,
                                  rangeslider=dict(visible=False)),
                       yaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False),
                       width=500,
                       height=500,
                       paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(data=[go.Candlestick(x=np.linspace(1,l,l),
                                         open=data.Open,
                                         high=data.High,
                                         low=data.Low,
                                         close=data.Close)],
                    layout=layout)
    fig.write_image("images/fig-53.png")

    # Convert to numpy array
    im = Image.open('images/fig-53.png')
    #im = im.resize((300,300),Image.ANTIALIAS)
    data = np.asarray(im)

    # Return the first channel of the image
    return data[:, :, 0]

In [62]:
START = datetime(1980, 1, 1)
END = datetime(2020, 4, 23)

In [63]:
tic = yf.Ticker('MSFT')
hist = tic.history(start=START, end=END)

In [64]:
def dta_transformation(data, est_h):
    # Make sure data has sufficient columns
    assert 'Open' in data.columns
    assert 'High' in data.columns
    assert 'Low' in data.columns
    assert 'Close' in data.columns

    data['lag_close'] = data['Close'].shift(1)
    data['Indicator'] = np.where(data['Close'] > data['lag_close'], 1, 0)

    x = []
    y = []
    for i in range(est_h, data.shape[0]):
        sub_dta = data.iloc[i - est_h:i]

        y_i = data.iloc[i]['Indicator']
        x_i = dta_to_candlestick(sub_dta)

        y.append(y_i)
        x.append(x_i)

        print("{}/{}".format(i - est_h, data.shape[0] - est_h))

    return x, y

In [65]:
x, y = dta_transformation(hist.iloc[1000:1040], 30)

C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Andyy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0/10
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10


In [66]:
x = np.stack(x, axis=2)

In [67]:
x.shape

(500, 500, 10)

In [68]:
im = Image.fromarray(x[:,:,0])
im.show()

In [46]:
x[:,:,0]

(100, 100)

In [20]:
sub_x = x[:,:,568:1568]

In [23]:
y = np.array(y)
y.shape

(8568,)

In [24]:
sub_y = y[568:1568]

In [25]:
np.savez_compressed('test_1', x=sub_x, y=sub_y)

In [27]:
x.shape[2] // 1000

8

In [28]:
for i in range(8):
    sub_x = x[:,:,(1000*i+568):(1000*(i+1)+568)]
    sub_y = y[(1000*i+568):(1000*(i+1)+568)]
    np.savez_compressed('test_{}'.format(i), x=sub_x, y=sub_y)

In [19]:
350000000/1024/1024

333.7860107421875

In [17]:
x.shape

2998800000

In [11]:
loaded = np.load('test_0.npz')

In [12]:
dta_x = loaded['x']

In [13]:
dta_x.shape

(500, 700, 1000)

In [14]:
dta_y = loaded['y']
dta_y.shape

(1000,)

## Pytorch Machine Learning Shit

In [1]:
from ResNet_CNN import *

In [2]:
import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision

In [3]:
from functools import partial
from collections import OrderedDict

In [4]:
import torchvision.models as models

In [5]:
from torchsummary import summary

In [6]:
model = res_conv1(1, 500)

In [7]:
print(model)

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(1, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBottleNeckBlock(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv2dAuto(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv2dAuto(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track

In [8]:
summary(model, (1,500,500))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 512, 32, 32]         --
|    └─Sequential: 2-1                   [-1, 32, 125, 125]        --
|    |    └─Conv2d: 3-1                  [-1, 32, 250, 250]        1,568
|    |    └─BatchNorm2d: 3-2             [-1, 32, 250, 250]        64
|    |    └─ReLU: 3-3                    [-1, 32, 250, 250]        --
|    |    └─MaxPool2d: 3-4               [-1, 32, 125, 125]        --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 512, 1, 1]           --
Total params: 549,435
Trainable params: 549,435
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 0.95
Forward/backward pass size (MB): 30.52
Params size (MB): 2.10
Estimated Total Size (MB

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 512, 32, 32]         --
|    └─Sequential: 2-1                   [-1, 32, 125, 125]        --
|    |    └─Conv2d: 3-1                  [-1, 32, 250, 250]        1,568
|    |    └─BatchNorm2d: 3-2             [-1, 32, 250, 250]        64
|    |    └─ReLU: 3-3                    [-1, 32, 250, 250]        --
|    |    └─MaxPool2d: 3-4               [-1, 32, 125, 125]        --
├─ResNetDecoder: 1-2                     [-1, 1]                   --
|    └─AdaptiveAvgPool2d: 2-2            [-1, 512, 1, 1]           --
Total params: 549,435
Trainable params: 549,435
Non-trainable params: 0
------------------------------------------------------------------------------------------
Input size (MB): 0.95
Forward/backward pass size (MB): 30.52
Params size (MB): 2.10
Estimated Total Size (MB

## Putting everything together

In [15]:
X = dta_x[:,:,568:768]

In [16]:
Y = dta_y[568:768]

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
import torchvision.transforms as transforms

In [21]:
trans = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
from torch.autograd import Variable
from torch.optim import Adam, SGD

In [24]:
X = X.reshape((-1, 500, 700)).astype(np.float32)
Y = Y.reshape((-1, 1)).astype(np.float32)

In [25]:
X.shape

(200, 500, 700)

In [26]:
Y.shape

(200, 1)

In [27]:
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.1, random_state=42)

In [28]:
(train_X.shape, train_Y.shape), (val_X.shape, val_Y.shape)

(((180, 500, 700), (180, 1)), ((20, 500, 700), (20, 1)))

In [29]:
train_X = train_X.reshape(-1, 1, 500, 700)
train_X = torch.from_numpy(train_X)

train_Y = train_Y.astype(int)
train_Y = torch.from_numpy(train_Y)

In [30]:
val_X =  val_X.reshape(-1, 1, 500, 700)
val_X = torch.from_numpy(val_X)

val_Y = val_Y.astype(int)
val_Y = torch.from_numpy(val_Y)

In [31]:
model = res_conv1(1, 500)

In [32]:
lr = 0.01
optimizer = Adam(model.parameters(), lr=lr)

criterion = nn.BCELoss()

In [33]:
def train(epochs):
    model.train()
    tr_loss = 0
    # dataset
    x_train, y_train = Variable(train_X), Variable(train_Y)
    x_val, y_val = Variable(val_X), Variable(val_Y)
 
    optimizer.zero_grad()
    
    output_train = model(x_train)
    output_val = model(x_val)
    
    loss_train = criterion(output_train, y_train.type(torch.float))
    loss_val = criterion(output_val, y_val.type(torch.float))
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    
    print('Epoch: ', epochs+1, '\t', 'train loss: ', loss_train, '\t', 'val loss: ', loss_val)

In [65]:
x_train, y_train = Variable(train_X), Variable(train_Y)
x_val, y_val = Variable(val_X), Variable(val_Y)
optimizer.zero_grad()

In [35]:
output_train = model(x_train)

In [36]:
output_val = model(x_val)

In [66]:
y_train = y_train.squeeze(1)

tensor([0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0], dtype=torch.int32)

In [72]:
loss_train = criterion(output_train, y_train.type(torch.float))
loss_val = criterion(output_val, y_val.type(torch.float))

In [74]:
loss_train.backward()

In [ ]:
n_epochs = 2
train_losses = []
val_losses = []

for epochs in range(n_epochs):
    train(epochs)

Epoch:  1 	 train loss:  tensor(0.6873, grad_fn=<BinaryCrossEntropyBackward>) 	 val loss:  tensor(0.6708, grad_fn=<BinaryCrossEntropyBackward>)
